# Data Collection & Cleaning
---
<img src="img/beautifulsoup.png" width="300" align="center"/>

In this notebook, by studying the HTML structure of a website, we will be using BeautifulSoup to extract the necessary data to build our movie dataset from scratch. There will be 3 stages of preparation:

> 1. To gather the well-known movie sites ratings 
> 2. To gather the box office numbers
> 3. To gather the film awards

In the meantime, we will also clean the data whenever it is necessary.

In [1]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import numpy as np
import re
pd.options.display.max_columns = None

## Stage 1: IMDb + Rotten Tomatoes + Metascore
---


### To scrape data from IMDb as base  

<img src="img/imdb.png" width="250" align="center"/>

We intend to collect the top 100 movies data ranked according to number of votes by IMDb users from year 1999 to 2019. So there will be 2,100 rows of data in the table. These are the types of data we can collect from the IMDb webpage:

> 1. Movie title
> 2. Year
> 3. Runtime in minutes
> 4. Certificate (PG-18, PG-15, etc.)
> 5. Genre 
> 6. IMDb ratings
> 7. IMDb number of votes
> 8. Metascore
> 9. Directors
> 10. Actors

In [2]:
def get_url_data(url):
    
    response = rq.get(url)
    html_soup = BeautifulSoup(response.content,"lxml")

    movie_containers = html_soup.find_all('div', {'class':'lister-item mode-advanced'})
    print(len(movie_containers))
    
    movie_dict = get_data(movie_containers)
    imdb_1 = pd.DataFrame(movie_dict)
    imdb_2 = imdb_1

    return imdb_2

In [3]:
def get_data(movie_containers):
    
    movie_dict = {'Movie':[], 'Year':[], 'Runtime':[], 'Certificate':[], 'Genre':[], 
                  'IMDb_rating':[], 'IMDb_votes':[], 'Metascore':[], 'Directors':[], 'Actors':[]}
    
    for first_movie in movie_containers:
        
        movie_dict['Movie'].append(first_movie.find('h3').find('a')
                                   .get_text() if first_movie.find('h3').find('a') else None)
        
        movie_dict['Year'].append(first_movie.find('h3')
                                  .find('span', {'class':'lister-item-year text-muted unbold'})
                                  .get_text() if first_movie.find('h3')
                                  .find('span', {'class':'lister-item-year text-muted unbold'}) else None)
        
        movie_dict['Runtime'].append(first_movie.find('p')
                                     .find('span', class_='runtime').get_text() if first_movie.find('p')
                                     .find('span', class_='runtime') else None)
        
        if first_movie.find('p').find('span', {'class':'certificate'}) is not None:
            movie_dict['Certificate'].append(first_movie.find('p').find('span', {'class':'certificate'}).get_text())
        else:
            movie_dict['Certificate'].append(None)
        
        movie_dict['Genre'].append(first_movie.find('p')
                                   .find('span', {'class':'genre'}).get_text()
                                   .split() if first_movie.find('p').find('span', {'class':'genre'}) else None)
        
        movie_dict['IMDb_rating'].append(first_movie
                                         .find('div', {'class':'inline-block ratings-imdb-rating'})
                                         .find('strong').get_text() if first_movie
                                         .find('div', {'class':'inline-block ratings-imdb-rating'}) else None)
        
        movie_dict['IMDb_votes'].append(first_movie.find_all('span', {'name':'nv'})[0]
                                        .get('data-value') if first_movie.find_all('span', {'name':'nv'}) else None)
        
        if first_movie.find('div', class_ = 'ratings-metascore') is not None:
            movie_dict['Metascore'].append(first_movie.find('div', {'class':'inline-block ratings-metascore'}).find('span').get_text().strip())
        else:
            movie_dict['Metascore'].append(None)
        
        directors = []
        actors=[]
        state = False
            
        for i in first_movie.find('p',class_='').find_all('a'):
            if state == False:
                directors.append(i.get_text('href'))
            else:
                actors.append(i.get_text('href'))
                        
            if i.next_sibling.next_sibling != None:
                if i.next_sibling.next_sibling.get('class') != None and len(i.next_sibling.next_sibling.get('class')) > 0:
                    state = i.next_sibling.next_sibling.get("class")[0] =='ghost'          
        
        movie_dict['Directors'].append(directors)
        movie_dict['Actors'].append(actors)

    return movie_dict

In [4]:
header = ['Movie', 'Year', 'Runtime', 'Certificate', 'Genre', 'IMDb_rating',
                'IMDb_votes', 'Metascore', 'Directors', 'Actors']
imdb = pd.DataFrame(columns=header)
imdb

,Movie,Year,Runtime,Certificate,Genre,IMDb_rating,IMDb_votes,Metascore,Directors,Actors


In [5]:
for year in range(1999,2020):
    url = ("https://www.imdb.com/search/title/?title_type=feature&release_date=%s&sort=num_votes,desc&count=100" %year)
    print(url)
    imdb_1 = get_url_data(url)
    imdb = imdb.append(imdb_1, ignore_index=True)

https://www.imdb.com/search/title/?title_type=feature&release_date=1999&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2000&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2001&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2002&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2003&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2004&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2005&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2006&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=feature&release_date=2007&sort=num_votes,desc&count=100
100
https://www.imdb.com/search/title/?title_type=

In [6]:
imdb  # As of 9 April 2020

,Movie,Year,Runtime,Certificate,Genre,IMDb_rating,IMDb_votes,Metascore,Directors,Actors
0,Fight Club,(1999),139 min,R(A),[Drama],8.8,1766938,66,[David Fincher],"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre..."
1,The Matrix,(1999),136 min,PG,"[Action,, Sci-Fi]",8.7,1592823,73,"[Lana Wachowski, Lilly Wachowski]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne..."
2,The Green Mile,(1999),189 min,R(A),"[Crime,, Drama,, Fantasy]",8.6,1080907,61,[Frank Darabont],"[Tom Hanks, Michael Clarke Duncan, David Morse..."
3,American Beauty,(1999),122 min,R(A),[Drama],8.3,1029619,84,[Sam Mendes],"[Kevin Spacey, Annette Bening, Thora Birch, We..."
4,The Sixth Sense,(1999),107 min,PG,"[Drama,, Mystery,, Thriller]",8.1,877505,64,[M. Night Shyamalan],"[Bruce Willis, Haley Joel Osment, Toni Collett..."
...,...,...,...,...,...,...,...,...,...,...
2095,The Farewell,(I) (2019),100 min,PG,"[Comedy,, Drama]",7.6,35486,89,[Lulu Wang],"[Shuzhen Zhao, Awkwafina, X Mayo, Hong Lu]"
2096,Child's Play,(2019),90 min,M18,"[Horror,, Sci-Fi]",5.8,34865,48,[Lars Klevberg],"[Tim Matheson, Ben Andrusco-Daon, Zahra Anders..."
2097,The Dirt,(2019),107 min,R21,"[Biography,, Comedy,, Drama]",7.0,34495,39,[Jeff Tremaine],"[Douglas Booth, Iwan Rheon, Machine Gun Kelly,..."
2098,Richard Jewell,(2019),131 min,NC16,"[Biography,, Crime,, Drama]",7.5,32568,68,[Clint Eastwood],"[Paul Walter Hauser, Sam Rockwell, Brandon Sta..."


In [7]:
imdb['Movie'] = imdb['Movie'].replace({'Gisaengchung': 'Parasite', 
                                       'Once Upon a Time... in Hollywood': 'Once Upon a Time in Hollywood'})

We have built the base movie dataset imdb containing 2100 rows and 10 columns of data. From now onwards we will be collecting data from other websites and then merge the data to our main movie dataset imdb. 


---
### To scrape Rotten Tomatoes and merge it to the main dataset
<img src="img/rotten tomatoes.jfif" width="350" align="center"/>

There are 2 types of data to be collected from Rotten Tomatoes:

> 1. Tomatometer (Rotten Tomatoes ratings)
> 2. Rotten Tomatoes number of reviews

In [8]:
#Function to extract rotten tomatoes rating from 1999-2019

def NET_rating(year_req, year_int, nameList):
    ratings = []
    titles = []
    complete_set=[]
    review = []
    i = 0
    j = 0
    unparsed = year_req.text
    bs_rate = BeautifulSoup(unparsed, 'lxml') #to parse for ratings
    bs_titles = BeautifulSoup(unparsed, 'html5lib') #to parse for titles
    
    #get titles
    table = bs_titles.find("table", "table")
    href = table.tbody.find_all("a")
    for a in href:
        label = a.text
        num_year = "("+str(year_int)+")"
        label = label.replace(num_year, "")
        label = label.replace("\n            ","")
        label = label.rstrip()
        titles.append(label)
    
    #get ratings
    scores = bs_rate.find_all("span", "tMeterIcon tiny")
    for b in scores:
        skore = b.find("span", class_="tMeterScore").text
        skore = skore.replace('\xa0', "")
        skore = int(skore.replace('%',''))
        ratings.append(skore)
        
    count = bs_rate.find_all("td", "right hidden-xs")
    for c in count:
        review.append(c.get_text("td"))

    #getting only the ratings for the movies that are in my file
    for d in titles:
        if(d in nameList):
            complete_set.append(ratings[i])
        else:
            complete_set.append(0)
        i+=1
        
    for e in titles:
        if(e in nameList):
            review.append(review[j])
        else:
            review.append(0)
        j+=1    
        
    return titles, complete_set, review

In [9]:
#changing our movie name column into a list
nameList = imdb["Movie"].tolist()

In [10]:
score_summary = []
review_summary=[]
title_summary = []
for year in range(1999,2020):
    req = rq.get('https://www.rottentomatoes.com/top/bestofrt/?year='+str(year))
    movNames, completeSet, review_ = NET_rating(req, year, nameList)
    score_summary.extend(completeSet)
    review_summary.extend(review_)
    title_summary.extend(movNames)

In [11]:
len(score_summary)

2100

In [12]:
len(title_summary)

2100

In [13]:
#appending the rotten tomatoes ratings
Rot_scores = []
for name in nameList:
    i = 0
    while(i<2100):
#checking if the titles under myMov["Title"] is in 
#the final list of movies gathered from rotten tomatoes
        if(name==title_summary[i]): 
            break
        i+=1
    if(i<2100):
#if found, append the rotten tomatoes rating 
        Rot_scores.append(score_summary[i])
    else:
#else, append in 0
        Rot_scores.append(0)

In [14]:
#appending the rotten tomatoes reviews
no_of_review = []
for name in nameList:
    i = 0
    while(i<2100):
#checking if the titles under myMov["Title"] is in 
#the final list of movies gathered from rotten tomatoes
        if(name==title_summary[i]): 
            break
        i+=1
    if(i<2100):
#if found, append the rotten tomatoes reviews 
        no_of_review.append(review_summary[i])
    else:
#else, append in 0
        no_of_review.append(0)

In [15]:
len(Rot_scores)

2100

In [16]:
len(no_of_review)

2100

In [17]:
#creating dataframe
imdb["RT_rating"] = pd.DataFrame(np.array(Rot_scores))
imdb["RT_review"] = pd.DataFrame(np.array(no_of_review))

In [18]:
imdb.tail(100)

,Movie,Year,Runtime,Certificate,Genre,IMDb_rating,IMDb_votes,Metascore,Directors,Actors,RT_rating,RT_review
2000,Joker,(2019),122 min,NC16,"[Crime,, Drama,, Thriller]",8.5,744537,59,[Todd Phillips],"[Joaquin Phoenix, Robert De Niro, Zazie Beetz,...",0,0
2001,Avengers: Endgame,(2019),181 min,PG13,"[Action,, Adventure,, Drama]",8.4,692582,78,"[Anthony Russo, Joe Russo]","[Robert Downey Jr., Chris Evans, Mark Ruffalo,...",94,283
2002,Once Upon a Time in Hollywood,(2019),161 min,M18,"[Comedy,, Drama]",7.7,430824,83,[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...",0,0
2003,Captain Marvel,(2019),123 min,PG13,"[Action,, Adventure,, Sci-Fi]",6.9,403426,64,"[Anna Boden, Ryan Fleck]","[Brie Larson, Samuel L. Jackson, Ben Mendelsoh...",78,68
2004,Parasite,(2019),132 min,M18,"[Comedy,, Drama,, Thriller]",8.6,354221,96,[Bong Joon Ho],"[Kang-ho Song, Sun-kyun Lee, Yeo-jeong Jo, Woo...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2095,The Farewell,(I) (2019),100 min,PG,"[Comedy,, Drama]",7.6,35486,89,[Lulu Wang],"[Shuzhen Zhao, Awkwafina, X Mayo, Hong Lu]",98,195
2096,Child's Play,(2019),90 min,M18,"[Horror,, Sci-Fi]",5.8,34865,48,[Lars Klevberg],"[Tim Matheson, Ben Andrusco-Daon, Zahra Anders...",0,0
2097,The Dirt,(2019),107 min,R21,"[Biography,, Comedy,, Drama]",7.0,34495,39,[Jeff Tremaine],"[Douglas Booth, Iwan Rheon, Machine Gun Kelly,...",0,0
2098,Richard Jewell,(2019),131 min,NC16,"[Biography,, Crime,, Drama]",7.5,32568,68,[Clint Eastwood],"[Paul Walter Hauser, Sam Rockwell, Brandon Sta...",0,0


### Splitting the Genre set into individual Genre (binary form)

In [19]:
temp = imdb.copy()
temp.head()
imdb = imdb.join(temp.Genre.apply(pd.Series).add_prefix('Genre_'))
imdb.head(3)

,Movie,Year,Runtime,Certificate,Genre,IMDb_rating,IMDb_votes,Metascore,Directors,Actors,RT_rating,RT_review,Genre_0,Genre_1,Genre_2
0,Fight Club,(1999),139 min,R(A),[Drama],8.8,1766938,66,[David Fincher],"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",78,171,Drama,NaN,NaN
1,The Matrix,(1999),136 min,PG,"[Action,, Sci-Fi]",8.7,1592823,73,"[Lana Wachowski, Lilly Wachowski]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",87,150,"Action,",Sci-Fi,NaN
2,The Green Mile,(1999),189 min,R(A),"[Crime,, Drama,, Fantasy]",8.6,1080907,61,[Frank Darabont],"[Tom Hanks, Michael Clarke Duncan, David Morse...",78,134,"Crime,","Drama,",Fantasy


### Cleaning the Data

In [20]:
### year brackets and punctions and convert to int
imdb['Year'] = imdb['Year'].astype(str).str.replace(r'\D+', '').astype(int)

### change runtime to runtime (min)
imdb['Runtime'] = imdb['Runtime'].astype(str).str.replace('min', '').astype(int)
imdb = imdb.rename(columns={'Runtime': 'Runtime (min)'})

### change rating to float
imdb['IMDb_rating'] = imdb['IMDb_rating'].fillna(0).astype(float)

### change metascore to int
imdb['Metascore'] = imdb['Metascore'].fillna(0).astype(int)

### change imdb_votes to int
imdb['IMDb_votes'] = imdb['IMDb_votes'].str.replace(',', '').fillna(0).astype(int)

### change RottenTomatoes rating to int
imdb['RT_rating'] = imdb['RT_rating'].fillna(0).astype(int)

### change RottenTomatoes review to int
imdb['RT_review'] = imdb['RT_review'].fillna(0).astype(int)

### remove list from director and join
imdb['Directors'] = imdb['Directors'].apply(lambda x: " ".join(x))

### remove ',' commas from genre
imdb['Genre_0'] = imdb['Genre_0'].str.replace(',', '')
imdb['Genre_1'] = imdb['Genre_1'].str.replace(',', '')
imdb['Genre_2'] = imdb['Genre_2'].str.replace(',', '')

In [21]:
imdb.head(3)

,Movie,Year,Runtime (min),Certificate,Genre,IMDb_rating,IMDb_votes,Metascore,Directors,Actors,RT_rating,RT_review,Genre_0,Genre_1,Genre_2
0,Fight Club,1999,139,R(A),[Drama],8.8,1766938,66,David Fincher,"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",78,171,Drama,NaN,NaN
1,The Matrix,1999,136,PG,"[Action,, Sci-Fi]",8.7,1592823,73,Lana Wachowski Lilly Wachowski,"[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",87,150,Action,Sci-Fi,NaN
2,The Green Mile,1999,189,R(A),"[Crime,, Drama,, Fantasy]",8.6,1080907,61,Frank Darabont,"[Tom Hanks, Michael Clarke Duncan, David Morse...",78,134,Crime,Drama,Fantasy


In [22]:
imdb.drop('Genre', axis=1, inplace=True)

In [23]:
imdb['Genre'] = imdb['Genre_0'].fillna('') + "," + imdb['Genre_1'].fillna('') + "," + imdb['Genre_2'].fillna('')


dummy_genre = imdb.Genre.str.get_dummies(',')

imdb = imdb.merge(dummy_genre, how = 'inner', left_index=True, right_index = True)

### change music and musical to one

imdb.loc[imdb['Music'] == 1, 'Musical'] = imdb['Music']
imdb.drop(columns='Music', inplace=True)

In [24]:
imdb.head(3)

,Movie,Year,Runtime (min),Certificate,IMDb_rating,IMDb_votes,Metascore,Directors,Actors,RT_rating,RT_review,Genre_0,Genre_1,Genre_2,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Fight Club,1999,139,R(A),8.8,1766938,66,David Fincher,"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",78,171,Drama,NaN,NaN,"Drama,,",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,The Matrix,1999,136,PG,8.7,1592823,73,Lana Wachowski Lilly Wachowski,"[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",87,150,Action,Sci-Fi,NaN,"Action,Sci-Fi,",1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,The Green Mile,1999,189,R(A),8.6,1080907,61,Frank Darabont,"[Tom Hanks, Michael Clarke Duncan, David Morse...",78,134,Crime,Drama,Fantasy,"Crime,Drama,Fantasy",0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0


In [25]:
imdb.drop(['Genre_0', 'Genre_1', 'Genre_2'], axis=1, inplace=True)
imdb.head(3)

,Movie,Year,Runtime (min),Certificate,IMDb_rating,IMDb_votes,Metascore,Directors,Actors,RT_rating,RT_review,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Fight Club,1999,139,R(A),8.8,1766938,66,David Fincher,"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",78,171,"Drama,,",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,The Matrix,1999,136,PG,8.7,1592823,73,Lana Wachowski Lilly Wachowski,"[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",87,150,"Action,Sci-Fi,",1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,The Green Mile,1999,189,R(A),8.6,1080907,61,Frank Darabont,"[Tom Hanks, Michael Clarke Duncan, David Morse...",78,134,"Crime,Drama,Fantasy",0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0


In [26]:
imdb.to_csv("imdb.csv", index=False)

---
# Stage 2: The Numbers (Box Office)

<img src="img/the numbers.jpg" width="400" align="center"/>

Pieces of information to be obtained:

> 1. Budget
> 2. Domestic (US) gross
> 3. Worldwide gross

In [27]:
from time import sleep
from random import randint

In [28]:
#budget_df = pd.DataFrame(budget_dict)
header = ['release_date', 'movie_name', 'budget', 'domestic_gross',
             'world_wide_gross']
numbers = pd.DataFrame(columns=header)

In [29]:
def get_budget_gross(list_tr):
    budget_dict = {'release_date':[], 'movie_name':[], 'budget':[], 
                       'domestic_gross':[], 'world_wide_gross':[]}
    for tr in list_tr:

        try:
            x_list = tr.find_all('td', {'class':'data'})
            #budget_dict['num'].append(x_list[0].get_text())  ### number
            budget_dict['release_date'].append(tr.find('a').get_text())  ### date
            budget_dict['movie_name'].append(tr.find('b').find('a').get_text())   ### movie name
            budget_dict['budget'].append(x_list[1].get_text())  ### budget
            budget_dict['domestic_gross'].append(x_list[2].get_text())  ### domestic_gross
            budget_dict['world_wide_gross'].append(x_list[3].get_text())  ### worldwide gross

        except:
            continue
    
    print(budget_dict)
    return  budget_dict

In [30]:
for i in range(1, 5700, 100):
    url = (f"https://www.the-numbers.com/movie/budgets/all/{i}")
    print(url)
    response = rq.get(url)
    html_soup = BeautifulSoup(response.content,"lxml")
    list_tr = html_soup.find('table').find_all('tr')
    
    budget_dict = get_budget_gross(list_tr)
    budget_iter_df = pd.DataFrame(budget_dict)
    numbers = numbers.append(budget_iter_df, ignore_index=True)
    
    sleep(randint(1,30))

https://www.the-numbers.com/movie/budgets/all/1
{'release_date': ['Apr 23, 2019', 'May 20, 2011', 'Apr 22, 2015', 'Dec 16, 2015', 'Apr 25, 2018', 'May 24, 2007', 'Nov 13, 2017', 'Oct 6, 2015', 'Dec 18, 2019', 'May 23, 2018', 'Mar 7, 2012', 'Mar 23, 2016', 'Jul 11, 2019', 'Nov 24, 2010', 'May 4, 2007', 'Apr 22, 2016', 'Jul 15, 2009', 'Dec 13, 2013', 'Dec 10, 2014', 'Apr 7, 2017', 'Nov 12, 2020', 'Dec 17, 2009', 'Jun 28, 2006', 'Jul 19, 2012', 'May 23, 2017', 'Nov 14, 2008', 'Apr 25, 2012', 'Jul 7, 2006', 'Jun 14, 2013', 'May 16, 2008', 'Jul 2, 2013', 'Jul 17, 2020', 'Jun 27, 2012', 'Apr 11, 2012', 'Jun 20, 2017', 'Jun 9, 2015', 'May 23, 2012', 'Jun 24, 2009', 'Jun 25, 2014', 'May 26, 2006', 'May 14, 2010', 'Dec 14, 2005', 'Dec 7, 2007', 'Feb 13, 2018', 'Dec 19, 1997', 'Dec 13, 2017', 'Jun 15, 2018', 'Dec 14, 2016', 'Jun 16, 2016', 'Jun 20, 2019', 'Jun 18, 2010', 'Apr 24, 2013', 'Apr 28, 2017', 'Jun 30, 2004', 'Mar 5, 2010', 'Oct 26, 2012', 'Dec 11, 2012', 'Jun 21, 2013', 'Mar 8, 2013', 

https://www.the-numbers.com/movie/budgets/all/101
{'release_date': ['Jul 16, 2018', 'May 18, 2016', 'Aug 9, 2018', 'May 28, 2014', 'Mar 6, 2019', 'Apr 7, 2016', 'Jun 12, 2015', 'Jul 5, 2017', 'Aug 2, 2016', 'May 28, 2009', 'Oct 27, 2017', 'Nov 21, 2018', 'Mar 27, 2009', 'Jun 16, 2017', 'Aug 7, 2009', 'Jun 30, 1999', 'Aug 1, 2008', 'Jun 22, 2007', 'Jul 28, 1995', 'Jan 8, 2020', 'May 11, 2017', 'Dec 6, 2013', 'Jun 6, 2018', 'Jul 31, 2014', 'May 7, 2010', 'Mar 28, 2014', 'Jul 9, 2014', 'Nov 10, 2004', 'May 30, 2012', 'Jul 1, 2003', 'May 5, 2004', 'Mar 27, 2019', 'May 29, 2019', 'May 20, 2015', 'Feb 1, 2019', 'May 20, 2016', 'May 21, 2010', 'Oct 24, 2016', 'Oct 25, 2014', 'Mar 26, 2010', 'Nov 1, 2012', 'Nov 5, 2014', 'Jun 22, 2016', 'Jul 29, 2011', 'Mar 16, 2017', 'Jun 27, 2019', 'Dec 6, 2018', 'May 17, 2007', 'Jul 16, 2010', 'Nov 18, 2015', 'May 24, 2013', 'May 14, 2014', 'Jun 3, 2011', 'Dec 25, 2008', 'Jul 14, 2010', 'May 12, 2006', 'May 25, 2016', 'Nov 1, 2019', 'Sep 29, 2016', 'Dec 10,

https://www.the-numbers.com/movie/budgets/all/201
{'release_date': ['Nov 16, 2007', 'May 9, 2012', 'Jun 28, 2013', 'Feb 12, 2010', 'Dec 15, 2016', 'Sep 25, 2015', 'Mar 11, 2011', 'Unknown', 'Nov 3, 2006', 'Jun 8, 2012', 'Dec 14, 2011', 'Jun 5, 2014', 'Jun 16, 1999', 'Feb 20, 2014', 'Nov 20, 2012', 'Apr 8, 2005', 'Jul 13, 2016', 'Nov 22, 2002', 'Jul 9, 2003', 'May 8, 2009', 'Jul 1, 1998', 'Jul 3, 2002', 'Jul 22, 2011', 'Jan 23, 2016', 'Jul 10, 1998', 'Mar 27, 2013', 'Dec 5, 2003', 'Dec 21, 2005', 'Dec 5, 2014', 'Jun 30, 2016', 'Oct 3, 2019', 'Feb 24, 2016', 'Dec 23, 2020', 'May 3, 2002', 'Mar 6, 2009', 'Jul 29, 2005', 'Jun 13, 2008', 'Jun 20, 2003', 'Jul 11, 2001', 'Nov 14, 2012', 'Mar 15, 2013', 'Dec 25, 2015', 'Nov 19, 1999', 'Mar 4, 2011', 'Nov 14, 2003', 'Jul 17, 2013', 'Jun 2, 2016', 'Nov 18, 2011', 'Jul 28, 2006', 'Oct 31, 2018', 'Jun 29, 2005', 'Nov 26, 2014', 'Nov 15, 2013', 'Jun 3, 2004', 'Aug 3, 2007', 'Jul 3, 2018', 'Jun 6, 2008', 'Jul 14, 2015', 'Mar 19, 2015', 'Dec 19, 2018

https://www.the-numbers.com/movie/budgets/all/301
{'release_date': ['May 24, 2000', 'Jun 30, 2000', 'Jul 20, 2016', 'Jun 15, 2007', 'Nov 21, 2012', 'Feb 16, 2007', 'Apr 9, 2018', 'Jun 27, 2003', 'Aug 9, 2013', 'Apr 19, 2013', 'Mar 24, 2017', 'Jul 19, 2002', 'Feb 12, 2014', 'Jun 11, 2004', 'Aug 5, 2015', 'May 9, 2008', 'Jul 22, 2005', 'Dec 17, 2010', 'Jun 23, 2010', 'Oct 2, 2018', 'May 19, 1999', 'May 19, 2005', 'May 16, 2002', 'Nov 2, 2001', 'Jul 26, 2013', 'Feb 7, 1997', 'Apr 6, 2016', 'Jun 14, 2002', 'Jul 4, 2008', 'Dec 24, 2010', 'Dec 8, 2017', 'May 11, 2018', 'Oct 3, 2013', 'Dec 22, 2006', 'Jun 10, 2005', 'May 27, 2015', 'Jul 29, 2011', 'Jun 27, 2007', 'Mar 18, 2015', 'Dec 10, 2004', 'Dec 19, 1997', 'Jun 28, 2000', 'Mar 7, 2014', 'Dec 17, 2004', 'Dec 21, 2016', 'Jan 14, 2011', 'Sep 21, 2016', 'Oct 7, 2011', 'Jun 12, 2019', 'Jun 11, 2010', 'Jul 31, 2013', 'Mar 9, 2016', 'Jun 12, 2009', 'Oct 25, 2013', 'Apr 2, 2004', 'Jun 13, 1997', 'May 6, 2005', 'Mar 23, 2017', 'Jun 16, 2004', 'Dec

https://www.the-numbers.com/movie/budgets/all/401
{'release_date': ['Dec 15, 2006', 'Jul 23, 2014', 'Dec 10, 2010', 'Nov 24, 1999', 'Dec 25, 2019', 'Jun 11, 2004', 'Jun 8, 2007', 'Dec 8, 2006', 'Nov 7, 2019', 'Sep 24, 2010', 'Nov 7, 1997', 'Jun 5, 2009', 'Dec 18, 2003', 'Mar 15, 2019', 'Jul 23, 2004', 'Aug 14, 2014', 'Nov 27, 2002', 'Mar 12, 2010', 'Oct 13, 2017', 'Dec 2, 2015', 'Dec 17, 2019', 'Nov 26, 2015', 'Apr 18, 2014', 'Feb 18, 2005', 'Dec 5, 2018', 'Aug 16, 2002', 'Jan 20, 2012', 'May 29, 2020', 'Nov 5, 2015', 'Feb 6, 2019', 'Nov 21, 2018', 'May 4, 2001', 'Nov 14, 2019', 'Dec 20, 2002', 'May 4, 2017', 'Mar 6, 2015', 'Jun 27, 2012', 'Dec 28, 2001', 'May 27, 2010', 'Feb 12, 2010', 'Aug 10, 2012', 'Nov 12, 2010', 'May 9, 1997', 'Mar 31, 2000', 'Dec 11, 2009', 'Jun 30, 2016', 'Dec 19, 2014', 'May 30, 2003', 'Dec 17, 2003', 'Dec 18, 2002', 'Jun 15, 2001', 'Jan 29, 2015', 'Jul 18, 2001', 'Aug 5, 2011', 'Feb 14, 2008', 'Oct 22, 2004', 'Feb 6, 2013', 'Apr 9, 2004', 'Dec 22, 1995', 'Dec

...

https://www.the-numbers.com/movie/budgets/all/5101
{'release_date': ['Oct 16, 2016', 'Nov 23, 2007', 'Unknown', 'Unknown', 'Oct 6, 2015', 'Jan 19, 2016', 'Sep 30, 2011', 'Apr 8, 1964', 'May 21, 1982', 'Aug 15, 1946', 'Aug 2, 1967', 'Dec 17, 1973', 'Aug 11, 1972', 'Feb 6, 2015', 'Jul 13, 2018', 'Dec 25, 1962', 'Mar 9, 2012', 'Oct 8, 1999', 'Feb 9, 2007', 'Apr 7, 2016', 'Jun 13, 1962', 'Aug 30, 1940', 'Nov 11, 1947', 'Sep 25, 1961', 'Dec 31, 1986', 'Nov 26, 2003', 'Jun 26, 1998', 'Jun 11, 2010', 'Oct 10, 2014', 'Aug 15, 2003', 'Oct 6, 2017', 'Aug 23, 2017', 'Jul 2, 2004', 'Mar 30, 2001', 'Dec 20, 2012', 'Aug 20, 2003', 'Jan 23, 2004', 'Oct 8, 2010', 'May 24, 1938', 'Jun 17, 2005', 'Aug 26, 2015', 'Nov 10, 2006', 'Mar 3, 2000', 'Sep 17, 2015', 'Aug 2, 2002', 'Jun 8, 2018', 'Jan 20, 2017', 'Dec 25, 2008', 'Jun 28, 2006', 'Mar 1, 1992', 'Jan 29, 1993', 'Oct 4, 2006', 'May 2, 2008', 'Sep 12, 2003', 'May 12, 2000', 'Aug 7, 2015', 'Jul 27, 2007', 'Apr 25, 2014', 'Apr 30, 1999', 'Sep 7, 2007', 

https://www.the-numbers.com/movie/budgets/all/5201
{'release_date': ['Oct 26, 2012', 'Dec 11, 2015', 'Apr 28, 1971', 'Unknown', 'Oct 14, 2008', 'Jul 7, 2007', 'Aug 12, 1927', 'Dec 24, 1947', 'Mar 19, 2013', 'Mar 30, 2012', 'Unknown', 'Feb 8, 2013', 'Jul 9, 2013', 'Unknown', 'Unknown', 'Oct 16, 1966', 'Sep 23, 2008', 'Mar 20, 2015', 'Unknown', 'Aug 21, 2015', 'Oct 16, 2015', 'Jul 5, 2016', 'Dec 1, 2015', 'Sep 24, 2010', 'Apr 21, 2015', 'Feb 8, 1976', 'May 8, 1987', 'Jul 25, 2008', 'Oct 8, 2010', 'Jul 11, 2003', 'Apr 24, 2015', 'Jun 3, 2011', 'Apr 9, 2010', 'Feb 8, 2013', 'Jun 29, 2007', 'Jul 26, 2013', 'Aug 14, 2015', 'Aug 27, 2010', 'Nov 16, 1976', 'Nov 9, 1984', 'Jun 27, 2012', 'Jan 29, 1964', '1973', 'Sep 18, 1951', 'Aug 3, 1968', 'Nov 15, 2002', 'Jun 15, 2001', 'Aug 23, 2011', 'Apr 22, 2011', 'Jan 7, 2005', '1993', 'Unknown', 'Mar 13, 2007', 'Feb 3, 2012', 'Unknown', 'Mar 11, 2014', 'Oct 20, 2015', 'Jan 31, 2020', 'Jun 18, 1999', 'Feb 7, 2003', 'Oct 8, 2010', 'Apr 23, 2019', 'Nov 23

https://www.the-numbers.com/movie/budgets/all/5301
{'release_date': ['Sep 23, 1994', 'Jul 26, 2013', 'Feb 1, 2013', 'Jun 26, 1998', 'Mar 2, 1984', 'Oct 21, 2016', 'Jan 18, 1985', 'Apr 19, 2002', 'Apr 19, 2013', 'Sep 16, 2016', 'Mar 25, 2005', 'Apr 22, 2016', 'Aug 31, 1994', 'May 17, 2002', 'Mar 7, 2008', 'Aug 10, 2001', 'Feb 11, 2011', 'Sep 5, 2007', 'Aug 19, 2011', 'Sep 7, 2007', 'Oct 31, 2008', 'Aug 6, 2010', 'Feb 5, 1936', 'May 11, 2007', 'Oct 25, 2013', 'Mar 22, 2002', 'Mar 6, 2015', 'Sep 28, 2005', 'Oct 9, 2009', 'May 29, 2009', 'Sep 28, 2015', 'Oct 17, 2014', 'Jul 20, 2018', 'Nov 4, 2005', 'Apr 17, 2014', 'Mar 17, 2006', 'Jun 3, 2011', 'Nov 11, 2014', 'Feb 13, 2015', 'May 26, 2015', 'Nov 20, 2015', 'Sep 8, 2015', 'Dec 21, 1937', '1968', 'Feb 12, 2016', 'Oct 4, 2018', 'Jun 19, 2015', 'Apr 29, 2005', 'Jun 25, 2014', 'Sep 17, 2004', 'Apr 5, 2013', 'May 1, 2010', 'Unknown', 'Unknown', 'Jul 6, 1966', 'Mar 5, 1999', 'Aug 19, 1983', 'Jun 26, 1936', 'Oct 9, 1998', 'Oct 11, 1996', 'May 22

https://www.the-numbers.com/movie/budgets/all/5401
{'release_date': ['Nov 19, 1982', 'Mar 21, 2003', 'Jun 1, 2012', 'Feb 6, 2018', 'Feb 11, 2011', 'Sep 18, 2015', 'Unknown', 'Oct 20, 1936', 'Jun 18, 2004', 'Aug 28, 2012', 'Sep 7, 2012', 'Sep 14, 2012', 'May 27, 1970', 'Feb 3, 2015', 'Unknown', 'Dec 6, 2019', 'Mar 21, 2014', 'Oct 7, 2005', 'May 12, 2006', 'Feb 1, 1980', 'Dec 25, 2005', 'Feb 11, 2005', 'Mar 22, 2012', 'Sep 26, 1986', 'Sep 4, 1987', 'Feb 22, 2002', 'Jun 23, 2017', 'May 18, 2012', 'May 14, 2003', 'Nov 21, 2014', 'May 20, 2016', 'Apr 7, 2009', 'Mar 23, 2010', 'Mar 10, 1972', 'Unknown', 'Unknown', 'Dec 11, 2009', 'Jan 23, 1943', 'Nov 21, 1976', 'Jan 6, 2012', 'Oct 7, 1963', 'Apr 17, 2015', 'May 24, 2006', 'Aug 23, 2013', 'May 24, 2012', 'Apr 10, 1981', 'May 8, 1963', 'Sep 18, 1987', 'Mar 11, 2016', 'Sep 16, 2005', 'Aug 18, 2000', 'Dec 29, 2010', 'Jan 20, 2006', 'Jun 17, 1970', 'Nov 15, 1956', 'Jul 19, 2018', 'Dec 14, 1949', 'Sep 24, 2015', 'Feb 3, 2017', 'Oct 19, 2012', 'Apr

https://www.the-numbers.com/movie/budgets/all/5501
{'release_date': ['Apr 8, 2016', 'Sep 24, 2004', 'Dec 30, 2002', 'Jul 7, 2006', 'Aug 22, 2001', 'Jul 31, 2015', 'Oct 30, 2009', 'Jun 1, 2012', 'Mar 11, 2016', 'Sep 16, 2005', 'Jul 1, 2011', 'Jul 29, 2016', 'Sep 25, 1998', 'Aug 7, 1998', 'Aug 31, 2001', 'Feb 13, 2015', 'Sep 22, 2000', 'Sep 3, 2010', 'Jan 17, 2007', 'Oct 17, 2014', 'Nov 21, 2008', 'Apr 10, 2015', 'Mar 19, 1999', 'Mar 18, 2005', 'May 15, 2015', 'Apr 21, 2006', 'Dec 13, 2002', 'Nov 22, 2013', 'Dec 11, 2015', 'Aug 14, 2015', 'Unknown', 'Dec 1, 2009', 'Sep 15, 2006', 'Oct 24, 2003', 'Jul 17, 2009', 'Sep 23, 2005', '2008', 'Jan 29, 2010', 'Mar 20, 2012', 'Oct 9, 2012', 'May 14, 2013', 'Jan 7, 2014', 'Oct 4, 2013', 'Oct 8, 2013', 'Unknown', 'Jun 24, 2014', 'Feb 10, 2015', 'Mar 10, 2015', 'Unknown', 'Apr 8, 2016', 'Mar 1, 2016', 'Oct 27, 2015', 'Jul 5, 2016', 'Dec 22, 2015', 'Nov 16, 2018', 'Feb 27, 2018', 'Jun 1, 2007', 'Apr 14, 2006', 'Sep 27, 2002', 'Sep 21, 2007', 'Jan 10, 

In [31]:
numbers.head(3)

,release_date,movie_name,budget,domestic_gross,world_wide_gross
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963"


In [32]:
numbers.to_csv('numbers.csv', index=False)

---
# Stage 3: Film Awards
<img src="img/wiki.jpg" width="400" align="center"/>

Data to be collected from different Wikipedia webpage:

> 1. winner of the film awards
> 2. nominees of the film awards

### Director Guild Awards (DGA)
<img src="img/dga.jpg" width="300" align="center"/>

In [45]:
dga_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Directors_Guild_of_America_Award_for_Outstanding_Directing_%E2%80%93_Feature_Film').text, 'lxml')

dga_results = []
current_year = 1
for table in dga_soup.find_all('table', {'class': 'wikitable'}):
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) == 4:
            current_year = int(re.search('[\d]{4}', columns[0].text).group(0))
            film_col = columns[2]
        else:
            film_col = columns[1]
        if columns[1].get('style') == 'background:#FAEB86;':
            winner = 1
        else:
            winner = 0
        try:
            a = film_col.find('i').find('a')
            dga_results.append((current_year, a.get('title'), winner))
        except:
            print(f"Problem with {row}")
            traceback.print_exc()

pd.DataFrame(dga_results, columns = ['year','film','winner']).to_csv('DGA.csv', index = False)

### British Academy Film Awards (BAFTA)
<img src="img/bafta.jpg" width="400" align="center"/>

In [46]:
bafta_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/BAFTA_Award_for_Best_Film').text, 'lxml')

bafta_results = []
current_year = 1
for table in bafta_soup.find_all('table', {'class': 'wikitable'})[2:]:
    year = 1947
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) == 1:
            if current_year != 2019: 
                # 2019 throws an error because a winner has not been picked as of 2/1
                current_year = int(re.search('[\d]{4}', columns[0].text).group(0))
            continue
        elif len(columns) == 5:
            film_col = columns[1]
        elif len(columns) == 4:
            film_col = columns[0]
        else:
            print(f"Wrong number of columns in {row}")
            
        winner = film_col.get('style') == 'background:#ccc;'
        if winner == True:
            winner = 1
        else:
            winner = 0
        try:
            a = film_col.find('a')
            bafta_results.append((current_year, a.get('title'), winner))
        except:
            print(f"Problem with {row}")
            
            traceback.print_exc()
pd.DataFrame(bafta_results, columns = ['year','film','winner']).to_csv('BAFTA.csv', index = False)

### Producer Guild Awards (PGA)
<img src="img/pga.jpg" width="400" align="center"/>

In [47]:
pga_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Producers_Guild_of_America_Award_for_Best_Theatrical_Motion_Picture').text, 'lxml')

pga_results = []
current_year = 1
for table in pga_soup.find_all('table', {'class': 'wikitable'}):
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) == 4:
            current_year = int(re.search('[\d]{4}', columns[0].text).group(0))
            film_col = columns[1]
        else:
            film_col = columns[0]
        if columns[1].get('style') == 'background:#FAEB86;':
            winner = 1
        else:
            winner = 0
        try:
            if film_col.find('i') is not None:
                a = film_col.find('i').find('a')
                pga_results.append((current_year, a.get('title'), winner))
        except:
            traceback.print_exc()

pd.DataFrame(pga_results, columns = ['year','film','winner']).to_csv('PGA.csv', index = False)

### Golden Globes Awards - Drama & Comedy
<img src="img/gg.jpg" width="400" align="center"/>

In [48]:
globes_drama_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Motion_Picture_%E2%80%93_Drama').text, 'lxml')

globes_drama_results = []
current_year = 1
for table in globes_drama_soup.find_all('table', {'class': 'wikitable'}):
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) == 4:
            current_year = columns[0].text.split('[')[0]
            film_col = columns[1]
        else:
            film_col = columns[0]
        if columns[1].get('style'):
            winner = 1
        else:
            winner = 0
        a = film_col.find('i').find('a')
        globes_drama_results.append((current_year, a.get('title'), winner))

pd.DataFrame(globes_drama_results, columns = ['year','film','winner']).to_csv('GG_drama.csv', index = False)

In [49]:
globes_comedy_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Motion_Picture_%E2%80%93_Musical_or_Comedy').text, 'lxml')

globe_comedy_results = []
current_year = 1
for table in globes_comedy_soup.find_all('table', {'class': 'wikitable'}):
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) == 4:
            current_year = columns[0].text.split('[')[0]
            film_col = columns[1]
        else:
            film_col = columns[0]
        if columns[1].get('style'):
            winner = 1
        else:
            winner = 0
        try:
            if film_col.find('i') is not None:
                a = film_col.find('i').find('a')
                globe_comedy_results.append((current_year, a.get('title'), winner))
        except:
            traceback.print_exc()

pd.DataFrame(globe_comedy_results, columns = ['year','film','winner']).to_csv('GG_comedy.csv', index = False)

### Cannes Film Festival - Palme d'Or (Golden Palm)
<img src="img/cannes.png" width="400" align="center"/>

In [50]:
GPalm_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Palme_d%27Or').text, 'lxml')
GPalm_elements = GPalm_soup.find('div', {'id': 'Palme_d&#039;Or_winning_films'}).findNext('ul').find_all('li')

GPalm_elements = GPalm_soup.find('div', {'id': 'Palme_d&#039;Or_winning_films'}).findNext('ul').find_all('li')
winners = dict()
for wel in GPalm_elements:
    year = int(re.search('[\d]{4}', wel.text).group(0))
    a = wel.find('a')
    href = a.get('href')
    title = a.get('title')
    winners[href] = (year, title)
table_years = set([1991, 1993, 1994] + list(range(2007, 2020)))
GPalm_results = []
for year in range(1970, 2020):
    soup = BeautifulSoup(rq.get(f'https://en.wikipedia.org/wiki/{year}_Cannes_Film_Festival').text, 'lxml')
    tag = next(x for x in soup.find_all('span', {'class': 'mw-headline'}) if x.text.lower().startswith('in competition'))
    if not tag:
        raise
    if year in table_years:
        elements = tag.findNext('tbody').find_all('tr')[1:]
    else:
        elements = tag.findNext('ul').find_all('li')
    for el in elements:
        a = el.findNext('a')
        winner = href in winners
        GPalm_results.append((year, title, winner))
GPalm = pd.DataFrame(GPalm_results, columns = ['year','film','winner'])

In [51]:
GPalm['winner'] = GPalm['winner']*1

In [52]:
GPalm.to_csv('GPalm.csv', index = False)

### Critics' Choice Movie Awards (CCMA)
<img src="img/ccma.jpg" width="400" align="center"/>

In [53]:
CCMA_result=list()

for i in range(1999,2020):
    
    url="https://www.filmaffinity.com/en/awards.php?award_id=critics_choice_awards&year=%s" %(i)
    page_soup = BeautifulSoup(rq.get(url).content,'lxml')
    
    for tag in page_soup.find_all('div',class_='aw-mc2 winner-border'):
        movie=tag.find('a').get('title')
        winner=1
        year=url[-4:]
        CCMA_result.append((movie, winner, year))
        
    for tag in page_soup.find_all('div',class_="aw-mc2"):
        for i in tag.find_all('a'):
            movie=i.get('title')
            winner=0
            year=url[-4:]
            CCMA_result.append((year, movie, winner))
            
fields = ['year','film','winner']
CCMA = pd.DataFrame(CCMA_result,columns=fields).to_csv('CCMA.csv', index=False)

### Academy Award - Best Picture  ( • ̀ω•́ )✧
<img src="img/oscar.jpg" width="500" align="center"/>

- the y value (respondent) to be predicted.  

In [54]:
oscar_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture').text, 'lxml')

oscar_results = []
current_year = 1
for table in oscar_soup.find_all('table', {'class': 'wikitable'}):
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) == 1:
            current_year = int(re.search('[\d]{4}', columns[0].text).group(0))
        elif len(columns) == 2:
            film_col = columns[0]
            if row.get('style') == 'background:#FAEB86':
                winner = 1
            else:
                winner = 0
            try:
                a = film_col.find('i').find('a')
                oscar_results.append((current_year, a.get_text('title'), winner))
            except:
                traceback.print_exc()
        else:
            continue

oscar = pd.DataFrame(oscar_results, columns = ['year','film', 'oscar_winner'])

In [55]:
oscar['oscar_nomination'] = 1
oscar

,year,film,oscar_winner,oscar_nomination
0,1927,Wings,1,1
1,1927,The Racket,0,1
2,1927,7th Heaven,0,1
3,1928,The Broadway Melody,1,1
4,1928,Alibi,0,1
...,...,...,...,...
558,2019,Joker,0,1
559,2019,Little Women,0,1
560,2019,Marriage Story,0,1
561,2019,1917,0,1


In [56]:
oscar.to_csv('oscar.csv', index=False)

--- 

That's the end of this notebook. Next, we will proceed to the section of Data Merging & Cleaning in which we will be merging all the separated csv files created in this notebook to the main dataset, and hence finalizing the movie dataset to be used for the rest of the project. ( • ̀ω•́ )✧